In [1]:
import os
import pyvtk
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

In [2]:
# The data structure in element-wise output is too complicated for xarray.open_mfdataset.
# Here we open the files as individual datasets and concatenate them on the variable level.
# This code is compatible with parallel netcdf build (single file output)

# load_wave_data=True:  read wave data and return numpy.ndarray
# load_wave_data=False: do not read wave data and return xarray.DataArray (use False if data is big)

def read_element_output(data_dir, load_wave_data=True):
    ################ open files ################
    # filenames
    nc_fnames = [f for f in os.listdir(data_dir) if 'axisem3d_synthetics.nc' in f]
    print('files to open: ', nc_fnames)

    # open files
    nc_files = []
    for nc_fname in nc_fnames:
        nc_files.append(xr.open_dataset(data_dir + '/' + nc_fname))
    
    ################ variables that are the same in the datasets ################
    # read Na grid (all azimuthal dimensions)
    na_grid = nc_files[0].data_vars['list_na_grid'].values.astype(int)

    # read time
    data_time = nc_files[0].data_vars['data_time'].values
    
    
    ################ variables to be concatenated over the datasets ################
    # define empty lists of xarray.DataArray objects
    xda_list_element_na = []
    xda_list_element_coords = []
    dict_xda_list_element = {}
    dict_xda_data_wave = {}
    for nag in na_grid:
        dict_xda_list_element[nag] = []
        dict_xda_data_wave[nag] = []
    
    # loop over nc files
    for nc_file in nc_files:
        # append DataArrays
        xda_list_element_na.append(nc_file.data_vars['list_element_na'])
        xda_list_element_coords.append(nc_file.data_vars['list_element_coords'])
        for nag in na_grid:
            dict_xda_list_element[nag].append(nc_file.data_vars['list_element__NaG=%d' % nag])
            dict_xda_data_wave[nag].append(nc_file.data_vars['data_wave__NaG=%d' % nag])
            
    # concat xarray.DataArray
    xda_list_element_na = xr.concat(xda_list_element_na, dim='dim_element')
    xda_list_element_coords = xr.concat(xda_list_element_coords, dim='dim_element')
    for nag in na_grid:
        dict_xda_list_element[nag] = xr.concat(dict_xda_list_element[nag], dim='dim_element__NaG=%d' % nag)
        dict_xda_data_wave[nag] = xr.concat(dict_xda_data_wave[nag], dim='dim_element__NaG=%d' % nag)
        
    # read data to numpy.ndarray
    list_element_na = xda_list_element_na.values.astype(int)
    list_element_coords = xda_list_element_coords.values
    dict_list_element = {}
    dict_data_wave = {}
    for nag in na_grid:
        dict_list_element[nag] = dict_xda_list_element[nag].values.astype(int)
        if load_wave_data:
            dict_data_wave[nag] = dict_xda_data_wave[nag].values
        
    ############### return ################
    if load_wave_data:
        return na_grid, data_time, list_element_na, list_element_coords, dict_list_element, dict_data_wave
    else:
        return na_grid, data_time, list_element_na, list_element_coords, dict_list_element, dict_xda_data_wave

# Inplane slices

## Read

In [3]:
# data dir
ModelName = 'model0012'
data_dir = './Runs/%s/output/elements/orthogonal_azimuthal_slices' %ModelName

# read
na_grid, data_time, list_element_na, list_element_coords, \
dict_list_element, dict_data_wave = read_element_output(data_dir)

files to open:  ['axisem3d_synthetics.nc.rank4', 'axisem3d_synthetics.nc.rank14', 'axisem3d_synthetics.nc.rank38', 'axisem3d_synthetics.nc.rank23', 'axisem3d_synthetics.nc.rank22', 'axisem3d_synthetics.nc.rank36', 'axisem3d_synthetics.nc.rank44', 'axisem3d_synthetics.nc.rank43', 'axisem3d_synthetics.nc.rank5', 'axisem3d_synthetics.nc.rank0', 'axisem3d_synthetics.nc.rank39', 'axisem3d_synthetics.nc.rank40', 'axisem3d_synthetics.nc.rank18', 'axisem3d_synthetics.nc.rank12', 'axisem3d_synthetics.nc.rank16', 'axisem3d_synthetics.nc.rank37', 'axisem3d_synthetics.nc.rank33', 'axisem3d_synthetics.nc.rank46', 'axisem3d_synthetics.nc.rank1', 'axisem3d_synthetics.nc.rank10', 'axisem3d_synthetics.nc.rank47', 'axisem3d_synthetics.nc.rank11', 'axisem3d_synthetics.nc.rank3', 'axisem3d_synthetics.nc.rank29', 'axisem3d_synthetics.nc.rank19', 'axisem3d_synthetics.nc.rank30', 'axisem3d_synthetics.nc.rank28', 'axisem3d_synthetics.nc.rank15', 'axisem3d_synthetics.nc.rank20', 'axisem3d_synthetics.nc.rank9',

In [5]:
from netCDF4 import Dataset
# wave dimension to animation
# wave_channel = 'z'
# wave_dim = 'spz'.index(wave_channel)

# time steps
ntime = len(data_time)

# phi of the slices
# phi_slices = [0.        , 0.01745329, 0.03490659, 0.05235988, 0.06981317, 0.08726646, 6.19591884, 6.21337214, 6.23082543, 6.24827872, 6.26573201]
# phi_slices= [0.        , 0.17453293, 0.34906585, 0.52359878, 0.6981317, 0.87266463, 1.04719755, 1.22173048, 1.3962634, 1.57079633, 1.74532925, 1.91986218, 2.0943951 , 2.26892803, 2.44346095, 2.61799388, 2.7925268 , 2.96705973, 3.14159265, 3.31612558, 3.4906585, 3.66519143, 3.83972435, 4.01425728, 4.1887902, 4.36332313, 4.53785606, 4.71238898, 4.88692191, 5.06145483, 5.23598776, 5.41052068, 5.58505361, 5.75958653, 5.93411946, 6.10865238]
phi_slices = np.radians(np.arange(0,360,1))
nslice = len(phi_slices)

# GLL coords on elements
nelem = list_element_coords.shape[0]
ngll = list_element_coords.shape[1]
# flattened coords, (s, z)
element_coords_sz = list_element_coords.reshape((nelem * ngll), 2)

# connectivity list, shared by all slices
# with GLL_points_one_edge = [0, 2, 4] in the inparam.output.yaml,
# the GLL point layout on each element is
# 0--1--2
# |  |  |
# 3--4--5
# |  |  |
# 6--7--8
connectivity = []


for ielem in np.arange(nelem):
    start = ielem * 9
    connectivity.append([start + 0, start + 1, start + 4, start + 3])
    connectivity.append([start + 1, start + 2, start + 5, start + 4])
    connectivity.append([start + 3, start + 4, start + 7, start + 6])
    connectivity.append([start + 4, start + 5, start + 8, start + 7])

# loop over slices
for islice, phi in enumerate(phi_slices):
    # create vtk folder
    # NETCDFDir = data_dir + '/netcdf/slice%d' % islice
    # os.makedirs(NETCDFDir, exist_ok=True)

    # nc = Dataset(NETCDFDir+'/%s/seismogram.nc' %EventName, 'w')
    # nc.createDimension('npoint', size=7199)
    
    # vtk mesh
    xyz = np.ndarray((nelem * ngll, 3))
    xyz[:, 0] = element_coords_sz[:, 0] * np.cos(phi)
    xyz[:, 1] = element_coords_sz[:, 0] * np.sin(phi)
    xyz[:, 2] = element_coords_sz[:, 1]

    # loop over elements to read wave data
    wave_s = np.ndarray((nelem * ngll, ntime))
    wave_p = np.ndarray((nelem * ngll, ntime))
    wave_z = np.ndarray((nelem * ngll, ntime))


    for ielem in np.arange(nelem):
        wave_s[(ielem * ngll):(ielem * ngll + ngll), :] = dict_data_wave[nslice][ielem, islice, :, 0, :]
        wave_p[(ielem * ngll):(ielem * ngll + ngll), :] = dict_data_wave[nslice][ielem, islice, :, 1, :]
        wave_z[(ielem * ngll):(ielem * ngll + ngll), :] = dict_data_wave[nslice][ielem, islice, :, 2, :]
        
    # loop over time to write netcdf
    for itime in np.arange(ntime):
        
        NETCDFDir = data_dir + '/netcdf/snapshot%d' % itime
        os.makedirs(NETCDFDir, exist_ok=True)

        nc = Dataset(NETCDFDir+'/slice%d.nc' %islice, 'w')
        nc.createDimension('npoint', size=len(xyz))
        nc.createDimension('3D', size=3)

        nc.createVariable('x', float, dimensions=('npoint'))
        nc['x'][:] = xyz[:,0]
        nc.createVariable('y', float, dimensions=('npoint'))
        nc['y'][:] = xyz[:,1]
        nc.createVariable('z', float, dimensions=('npoint'))
        nc['z'][:] = xyz[:,2]
        nc.createVariable('time', float, dimensions=('npoint'))
        nc['time'][:] = np.ones(len(xyz))*data_time[itime]

        # convert spz to xyz coordinate frame by formula 3.9 Leng thesis page 28
        nc.createVariable('disp_x', float, dimensions=('npoint'))
        nc['disp_x'][:] = wave_s[:,itime] * np.cos(phi) - wave_p[:,itime] * np.sin(phi)
        nc.createVariable('disp_y', float, dimensions=('npoint'))
        nc['disp_y'][:] = wave_p[:,itime] * np.sin(phi) + wave_p[:,itime] * np.cos(phi)
        nc.createVariable('disp_z', float, dimensions=('npoint'))
        nc['disp_z'][:] = wave_z[:,itime]

        nc.close()

        print('Done time step %d / %d' % (itime + 1, ntime), end='\r')
    print('\nDone slice %d / %d' % (islice + 1, len(phi_slices)))

Done time step 7 / 7
Done slice 1 / 360
Done time step 7 / 7
Done slice 2 / 360
Done time step 7 / 7
Done slice 3 / 360
Done time step 7 / 7
Done slice 4 / 360
Done time step 7 / 7
Done slice 5 / 360
Done time step 7 / 7
Done slice 6 / 360
Done time step 7 / 7
Done slice 7 / 360
Done time step 7 / 7
Done slice 8 / 360
Done time step 7 / 7
Done slice 9 / 360
Done time step 7 / 7
Done slice 10 / 360
Done time step 7 / 7
Done slice 11 / 360
Done time step 7 / 7
Done slice 12 / 360
Done time step 7 / 7
Done slice 13 / 360
Done time step 7 / 7
Done slice 14 / 360
Done time step 7 / 7
Done slice 15 / 360
Done time step 7 / 7
Done slice 16 / 360
Done time step 7 / 7
Done slice 17 / 360
Done time step 7 / 7
Done slice 18 / 360
Done time step 7 / 7
Done slice 19 / 360
Done time step 7 / 7
Done slice 20 / 360
Done time step 7 / 7
Done slice 21 / 360
Done time step 7 / 7
Done slice 22 / 360
Done time step 7 / 7
Done slice 23 / 360
Done time step 7 / 7
Done slice 24 / 360
Done time step 7 / 7
Done

## Generate animation on each slice

In [4]:
from netCDF4 import Dataset
# wave dimension to animation
# wave_channel = 'z'
# wave_dim = 'spz'.index(wave_channel)

# time steps
ntime = len(data_time)

# phi of the slices
# phi_slices = [0.        , 0.01745329, 0.03490659, 0.05235988, 0.06981317, 0.08726646, 6.19591884, 6.21337214, 6.23082543, 6.24827872, 6.26573201]
# phi_slices= [0.        , 0.17453293, 0.34906585, 0.52359878, 0.6981317, 0.87266463, 1.04719755, 1.22173048, 1.3962634, 1.57079633, 1.74532925, 1.91986218, 2.0943951 , 2.26892803, 2.44346095, 2.61799388, 2.7925268 , 2.96705973, 3.14159265, 3.31612558, 3.4906585, 3.66519143, 3.83972435, 4.01425728, 4.1887902, 4.36332313, 4.53785606, 4.71238898, 4.88692191, 5.06145483, 5.23598776, 5.41052068, 5.58505361, 5.75958653, 5.93411946, 6.10865238]
phi_slices = np.radians(np.arange(0,360,1))
nslice = len(phi_slices)

# GLL coords on elements
nelem = list_element_coords.shape[0]
ngll = list_element_coords.shape[1]
# flattened coords, (s, z)
element_coords_sz = list_element_coords.reshape((nelem * ngll), 2)

# connectivity list, shared by all slices
# with GLL_points_one_edge = [0, 2, 4] in the inparam.output.yaml,
# the GLL point layout on each element is
# 0--1--2
# |  |  |
# 3--4--5
# |  |  |
# 6--7--8
connectivity = []


for ielem in np.arange(nelem):
    start = ielem * 9
    connectivity.append([start + 0, start + 1, start + 4, start + 3])
    connectivity.append([start + 1, start + 2, start + 5, start + 4])
    connectivity.append([start + 3, start + 4, start + 7, start + 6])
    connectivity.append([start + 4, start + 5, start + 8, start + 7])

# loop over slices
for islice, phi in enumerate(phi_slices[0:1]):
    # create vtk folder
    # NETCDFDir = data_dir + '/netcdf/slice%d' % islice
    # os.makedirs(NETCDFDir, exist_ok=True)

    # nc = Dataset(NETCDFDir+'/%s/seismogram.nc' %EventName, 'w')
    # nc.createDimension('npoint', size=7199)
    
    # vtk mesh
    xyz = np.ndarray((nelem * ngll, 3))
    xyz[:, 0] = element_coords_sz[:, 0] * np.cos(phi)
    xyz[:, 1] = element_coords_sz[:, 0] * np.sin(phi)
    xyz[:, 2] = element_coords_sz[:, 1]

    # loop over elements to read wave data
    wave_s = np.ndarray((nelem * ngll, ntime))
    wave_p = np.ndarray((nelem * ngll, ntime))
    wave_z = np.ndarray((nelem * ngll, ntime))


    for ielem in np.arange(nelem):
        wave_s[(ielem * ngll):(ielem * ngll + ngll), :] = dict_data_wave[nslice][ielem, islice, :, 0, :]
        wave_p[(ielem * ngll):(ielem * ngll + ngll), :] = dict_data_wave[nslice][ielem, islice, :, 1, :]
        wave_z[(ielem * ngll):(ielem * ngll + ngll), :] = dict_data_wave[nslice][ielem, islice, :, 2, :]
        
    # loop over time to write netcdf
    for itime in np.arange(ntime):
        
        NETCDFDir = data_dir + '/netcdf/snapshot%d' % itime
        os.makedirs(NETCDFDir, exist_ok=True)

        nc = Dataset(NETCDFDir+'/slice%d.nc' %islice, 'w')
        nc.createDimension('npoint', size=len(xyz))
        nc.createDimension('3D', size=3)

        nc.createVariable('x', float, dimensions=('npoint'))
        nc['x'][:] = xyz[:,0]
        nc.createVariable('y', float, dimensions=('npoint'))
        nc['y'][:] = xyz[:,1]
        nc.createVariable('z', float, dimensions=('npoint'))
        nc['z'][:] = xyz[:,2]
        nc.createVariable('time', float, dimensions=('npoint'))
        nc['time'][:] = np.ones(len(xyz))*data_time[itime]

        # convert spz to xyz coordinate frame by formula 3.9 Leng thesis page 28
        nc.createVariable('disp_x', float, dimensions=('npoint'))
        nc['disp_x'][:] = wave_s[:,itime] * np.cos(phi) - wave_p[:,itime] * np.sin(phi)
        nc.createVariable('disp_y', float, dimensions=('npoint'))
        nc['disp_y'][:] = wave_p[:,itime] * np.sin(phi) + wave_p[:,itime] * np.cos(phi)
        nc.createVariable('disp_z', float, dimensions=('npoint'))
        nc['disp_z'][:] = wave_z[:,itime]

        nc.close()

        print('Done time step %d / %d' % (itime + 1, ntime), end='\r')
    print('\nDone slice %d / %d' % (islice + 1, len(phi_slices)))

Done time step 20 / 20
Done slice 1 / 360


In [ ]:
import shutil
# Move netcdf file
shutil.move(data_dir + '/netcdf/', './Runs/Sample/%s/snapshot' %ModelName)

In [8]:
print(data_dir + '/netcdf/', './Runs/Sample/%s/snapshot' %ModelName)

./Runs/globalmodel0013/output/elements/orthogonal_azimuthal_slices/netcdf/ ./Runs/Sample/globalmodel0013/snapshot
